Load Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

Define Transformations

In [58]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(    (0.5,0.5,0.5),(0.5,0.5,0.5)     )
])

Load the data

In [59]:
train_data = datasets.CIFAR10(root = './data',
                                    train = True,
                                   download = True,
                                   transform = transform)

test_data = datasets.CIFAR10(root = './data',
                                 train = False,
                                 download = True,
                                transform = transform)

In [60]:
train_loader = DataLoader(dataset = train_data , batch_size = 64 , shuffle = True)
test_loader = DataLoader(dataset = test_data , batch_size = 64 , shuffle = False)

Build the Architecture

In [64]:
class cnn_cifar(nn.Module):
    def __init__(self):
        super(cnn_cifar, self).__init__()
        self.cnn1 = nn.Conv2d(3, 6, 3)
       
        self.cnn2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2)

        x = self.cnn2(x)
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2)
        
        # flatten
        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = torch.relu(x)

        x = self.fc2(x)
        x = torch.relu(x)

        x = self.fc3(x)

        return x


In [65]:
model = cnn_cifar()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.0001)

Training

In [68]:
for epoch in range(10):
    for images, label in (train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch + 1}/10], loss : {loss.item():.4f}")
        

Epoch [1/10], loss : 0.9088
Epoch [2/10], loss : 1.4356
Epoch [3/10], loss : 1.1862
Epoch [4/10], loss : 1.8830
Epoch [5/10], loss : 1.9001
Epoch [6/10], loss : 1.2759
Epoch [7/10], loss : 0.8171
Epoch [8/10], loss : 1.4523
Epoch [9/10], loss : 1.1051
Epoch [10/10], loss : 0.8543


In [70]:

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images,labels in test_loader:
        outputs = model(images)

        _,predictions = torch.max(outputs,1)
        correct += (predictions==labels).sum().item()
        total += labels.size(0)
print(f"accuracy = {(correct/total)*100}")
#print(outputs[0].data)
#print(type(outputs[0].data))

accuracy = 56.874
